# Dual Unified Pipeline Demo
Runs WOE + RAW unified pipeline on synthetic data and prints a summary.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from risk_pipeline.core.config import Config
from risk_pipeline.unified_pipeline import UnifiedRiskPipeline

def generate_synthetic(n=4000, seed=42):
    rng = np.random.default_rng(seed)
    dt0 = datetime(2023,1,1)
    app_dt = [dt0 + timedelta(days=int(d)) for d in rng.integers(0,365,size=n)]
    app_id = np.arange(1, n+1)
    x1 = rng.normal(0,1,n)
    x2 = rng.exponential(1.0, n)
    x3 = rng.binomial(1, 0.3, n)
    cat = rng.choice(['A','B','C','D', None], size=n, p=[0.3,0.3,0.2,0.15,0.05])
    cat_map = {'A':0.2,'B':0.0,'C':-0.1,'D':0.3}
    cat_term = pd.Series(cat).map(cat_map).fillna(0).values
    logit = -1.0 + 0.8*x1 + 0.5*(x2>1.0).astype(int) + 0.6*x3 + cat_term
    p = 1/(1+np.exp(-logit))
    y = (rng.random(n) < p).astype(int)
    return pd.DataFrame({'app_id':app_id,'app_dt':app_dt,'x1':x1,'x2':x2,'x3':x3,'cat':cat,'target':y})

df = generate_synthetic()
cfg = Config(target_col='target', id_col='app_id', time_col='app_dt', enable_scoring=False, enable_calibration=True, stage2_method='lower_mean', enable_woe=True, selection_order=['psi','vif','correlation','iv','boruta','stepwise'], use_optuna=False, model_type='LogisticRegression', use_test_split=True, oot_months=3, equal_default_splits=True, n_risk_bands=10, band_method='quantile', enable_dual_pipeline=True)
pipe = UnifiedRiskPipeline(cfg)
results = pipe.fit(df)
results['selected_features'], results.get('best_model_name')
